
# Lab 5
**Duration:** ~2 hours

**Topics:**  
- `groupby()` with `mean/sum/count`; `agg()` multi-aggregation; multi-index results  
- `pivot_table()`; merging/joining DataFrames  
- Load & explore; clean missing/inconsistent data; derive new columns  
- Use `groupby`, filtering, sorting; visualize with Pandas `.plot()`; save outputs  
- Plots: line/bar/hist/pie via `.plot()`; customize labels/titles; plot from groupby  
- Intro to `matplotlib.pyplot` (`plot`, `bar`, `scatter`); styling & basic subplots

**Instructions:** For each task, read the markdown cell and implement your solution in the empty code cell directly below it.


# Case Study: Myntra Pants Dataset — Load & Explore

**Exercise 1:** Load the dataset from `myntra_dataset_ByScraping.csv` into `df`. Show `df.shape`, `df.head(3)`, and list column names.

In [30]:
import pandas as pd
df=pd.read_csv(r"myntra_dataset_ByScraping.csv")
print(df.shape) # to get the shape of df
print(df.head(3)) # to get top 3 rows from df
print(df.columns.tolist()) # tolist() : to convert all columns in a list form

(52120, 7)
       brand_name           pants_description   price     MRP  \
0           WROGN  Men Loose Fit Cotton Jeans  1374.0  2499.0   
1  Flying Machine          Men Slim Fit Jeans  1829.0  2999.0   
2        Roadster       Men Pure Cotton Jeans   974.0  2499.0   

   discount_percent  ratings  number_of_ratings  
0              0.45      4.2               57.0  
1              0.39      4.6                5.0  
2              0.61      3.6             1100.0  
['brand_name', 'pants_description', 'price', 'MRP', 'discount_percent', 'ratings', 'number_of_ratings']


**Exercise 2:** Run `df.info()` and `df.describe(include=['number']))`. Briefly inspect possible numeric columns (price, MRP, discount_percent, ratings, number_of_ratings).

In [33]:
df.info()
df.describe(include=['number']) # include=['number'] : over only numeric columns
# adhaar number : 12 digit number df.describe(), numeric_only=True

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52120 entries, 0 to 52119
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   brand_name         52120 non-null  object 
 1   pants_description  52120 non-null  object 
 2   price              52120 non-null  float64
 3   MRP                52120 non-null  float64
 4   discount_percent   52120 non-null  float64
 5   ratings            52120 non-null  float64
 6   number_of_ratings  52120 non-null  float64
dtypes: float64(5), object(2)
memory usage: 2.8+ MB


,price,MRP,discount_percent,ratings,number_of_ratings
count,52120.000000,52120.000000,52120.000000,52120.000000,52120.000000
mean,1594.515445,3180.398438,1.648256,3.997794,91.568937
std,1495.972325,2201.883218,4.687529,0.420404,433.918513
min,337.000000,499.000000,0.020000,1.000000,5.000000
25%,989.000000,2499.000000,0.400000,3.800000,16.000000
50%,1439.000000,2999.000000,0.500000,4.000000,35.000000
75%,1829.000000,3499.000000,0.630000,4.200000,74.000000
max,54000.000000,72000.000000,64.000000,5.000000,30700.000000


# Cleaning: Missing & Inconsistent Data

**Exercise 1:** Display total missing values per column with `df.isnull().sum()`. Identify columns with missing values.

In [36]:
colmn_null=df.isnull().sum()
print(colmn_null>0)

brand_name           False
pants_description    False
price                False
MRP                  False
discount_percent     False
ratings              False
number_of_ratings    False
dtype: bool


**Exercise 2:** Ensure numeric columns are numeric: cast `price`, `MRP`, `discount_percent`, `ratings`, `number_of_ratings` to numeric (coerce errors).

In [39]:
num_col=["price","MRP","discount_percent","ratings","number_of_ratings"]
# using for loop : col : price 
# df[col(price)]=to_numeric(df[col(price),coerce: non numeric values > NAN)
for col in num_col :
    df[col]=pd.to_numeric(df[col],errors="coerce") # to_numeric : convertic data type into numeric, coerece (non numeric value : nan)
df

,brand_name,pants_description,price,MRP,discount_percent,ratings,number_of_ratings
0,WROGN,Men Loose Fit Cotton Jeans,57.0,57.0,57.0,57.0,57.0
1,Flying Machine,Men Slim Fit Jeans,5.0,5.0,5.0,5.0,5.0
2,Roadster,Men Pure Cotton Jeans,1100.0,1100.0,1100.0,1100.0,1100.0
3,Bene Kleed,Relaxed Fit Denim Jeans,4800.0,4800.0,4800.0,4800.0,4800.0
4,Levis,Men 511 Slim Fit Jeans,264.0,264.0,264.0,264.0,264.0
...,...,...,...,...,...,...,...
52115,Pepe Jeans,Men Slim Fit Jeans,63.0,63.0,63.0,63.0,63.0
52116,HERE&NOW,Men Slim Fit Jeans,63.0,63.0,63.0,63.0,63.0
52117,Pepe Jeans,Men Slim Fit Jeans,63.0,63.0,63.0,63.0,63.0
52118,Celio,Men Straight Fit Jeans,63.0,63.0,63.0,63.0,63.0


**Exercise 3:** Handle missing numeric data: fill `ratings` with its median and `number_of_ratings` with 0. Drop rows where `price` or `MRP` is missing.

In [41]:
df['ratings']=df['ratings'].fillna(df['ratings'].median()) # dealing with rating column with median
df['number_of_ratings']=df['number_of_ratings'].fillna(0) # number of ratings filled with zero
df=df.dropna(subset=['price','MRP']) # dropping na from MRP and Price
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52120 entries, 0 to 52119
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   brand_name         52120 non-null  object 
 1   pants_description  52120 non-null  object 
 2   price              52120 non-null  float64
 3   MRP                52120 non-null  float64
 4   discount_percent   52120 non-null  float64
 5   ratings            52120 non-null  float64
 6   number_of_ratings  52120 non-null  float64
dtypes: float64(5), object(2)
memory usage: 2.8+ MB


**Exercise 4:** Fix inconsistencies: ensure `price` and `MRP` are positive; clip negatives to NaN then drop. Ensure `discount_percent` is between 0 and 1 (clip), then recompute a clean `net_discount` = `(MRP - price) / MRP`.

In [42]:
df['price']=df['price'].clip(lower=0) # clip : lower limit
df["MRP"]=df["MRP"].clip(lower=0) 
# clip : changing negative values to 0, NAN
df=df.dropna(subset=['price','MRP']) # dropping na
df['discount_percent']=df['discount_percent'].clip(0,1) # cliiping between 0 and 1
df['net_discount']=(df["MRP"]-df["price"]) / df["MRP"] # recomputing net_discount
df

,brand_name,pants_description,price,MRP,discount_percent,ratings,number_of_ratings,net_discount
0,WROGN,Men Loose Fit Cotton Jeans,57.0,57.0,1.0,57.0,57.0,0.0
1,Flying Machine,Men Slim Fit Jeans,5.0,5.0,1.0,5.0,5.0,0.0
2,Roadster,Men Pure Cotton Jeans,1100.0,1100.0,1.0,1100.0,1100.0,0.0
3,Bene Kleed,Relaxed Fit Denim Jeans,4800.0,4800.0,1.0,4800.0,4800.0,0.0
4,Levis,Men 511 Slim Fit Jeans,264.0,264.0,1.0,264.0,264.0,0.0
...,...,...,...,...,...,...,...,...
52115,Pepe Jeans,Men Slim Fit Jeans,63.0,63.0,1.0,63.0,63.0,0.0
52116,HERE&NOW,Men Slim Fit Jeans,63.0,63.0,1.0,63.0,63.0,0.0
52117,Pepe Jeans,Men Slim Fit Jeans,63.0,63.0,1.0,63.0,63.0,0.0
52118,Celio,Men Straight Fit Jeans,63.0,63.0,1.0,63.0,63.0,0.0


# Deriving New Columns

**Exercise 1:** Create `discount_amount = MRP - price`.

In [ ]:
arr=np.array([1,3,2,5,4,7,6,9,8])
print(np.where(arr>6))
ar=np.sort(arr)
print(ar)
np.where(ar>6)

**Exercise 2:** Bucket `ratings` into categories with `pd.cut`: ['low', 'mid', 'high'] using bins [0,3.5,4.2,5]. Store in `rating_band`.

**Exercise 3:** Create `review_band` from `number_of_ratings` using `pd.qcut` into 4 quantiles (Q1..Q4).

**Exercise 4:** Create a simple `value_score = ratings * (1 + net_discount)`.

# GroupBy, Aggregations, Multi-index

**Exercise 1:** Group by `brand_name` and compute `mean price`, `mean ratings`, and `count` of products.

**Exercise 2:** Use `agg()` for multi-aggregation on `price` and `ratings` grouped by `brand_name` (mean, median, std).

**Exercise 3:** Create a **multi-index** groupby by `brand_name` and `rating_band`, aggregating `price` mean and product count. Display the first 10 rows.

# Pivot Tables

**Exercise 1:** Build a pivot table with index=`brand_name`, columns=`rating_band`, values=`price`, `aggfunc='mean'`.

**Exercise 2:** Create a pivot table of product counts by `brand_name` (rows) and `review_band` (columns).

# Merging / Joining DataFrames

**Exercise 1:** Create a small mapping DataFrame `brand_segment` with two segments: mark the **top 5 brands by product count** as 'Top', others as 'Other'. Merge it back to `df` on `brand_name`.

**Exercise 2:** Compute a summary DataFrame `seg_summary` = mean `price` and mean `ratings` by `segment`, and sort by mean price desc.

# Filtering & Sorting

**Exercise 1:** Filter products with `value_score >= 4.0` and `net_discount >= 0.3`. Sort by `value_score` desc; show top 10.

# Visualizing with Pandas .plot() (line, bar, hist, pie); Plotting from groupby

**Exercise 1:** From `gb_brand` (mean_price/count), plot a **bar chart** of the top 10 brands by `count_items`. Add labels and title.

**Exercise 2:** Plot a **histogram** of `net_discount` (bins=20). Label axes and add a title.

**Exercise 3:** From `seg_summary`, draw a **pie chart** of `count` by `segment` with percentage labels.

**Exercise 4:** Build a **line plot** showing mean `price` per `rating_band` (order low→mid→high).

**Exercise 5:** Plot directly from groupby: group by `brand_name` and plot mean `ratings` (top 10 brands by count). Use a horizontal bar chart.

# matplotlib.pyplot Basics: plot(), bar(), scatter(); Styling & Subplots

**Exercise 1:** Using matplotlib directly, create a **scatter** of `price` vs `ratings` for 500 random samples (if available). Add labels/title and alpha for visibility.

**Exercise 2:** Create a **bar chart** with matplotlib: show counts of products per `segment`.

**Exercise 3:** Create a 1x2 **subplot** figure: (left) histogram of `ratings`; (right) histogram of `price` (use 20 bins). Add overall suptitle.

# Save Cleaned Data & Figures

**Exercise 1:** Save cleaned DataFrame to csv` to your local system or drive.